# Imports

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests
import pandas

In [2]:
PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
driver = webdriver.Chrome(PATH)
url= "https://www.sustainalytics.com/esg-ratings/"
driver.maximize_window()
driver.get(url)

In [5]:
driver.find_element(By.ID(searchInput))

NameError: name 'searchInput' is not defined

In [47]:
content = driver.page_source.encode('utf-8').strip()
soup = BeautifulSoup(content,"html.parser")

# AMZN

In [48]:
company_details = soup.find(class_="row company-details d-flex")

In [49]:
risk_rating = company_details.find(class_="col-xs-6 risk-rating-score").text
print(risk_rating)

27.4


In [50]:
risk_category = company_details.find(class_="col-xs-6 risk-rating-assessment").text
print(risk_category)

Medium Risk


In [51]:
industry_group_position = (int(company_details.find(class_="industry-group-position").text) / int(company_details.find(class_="industry-group-positions-total").text)) * 100
print(industry_group_position)

99.77973568281938


In [52]:
overall_rank = (int(company_details.find(class_="universe-position").text) / int(company_details.find(class_="universe-positions-total").text)) * 100
print(overall_rank)

46.72632363907532
